In [ ]:
!cp -r /content/drive/MyDrive/Lenguaje/Proyecto/Modules Modules
!pip install top2vec

     |████████████████████████████████| 5.2 MB 6.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 58.3 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2341955 sha256=bd29563b7a3296e9555c0201062f3455f660ffe3cd5e647a33c90cf009d0a615
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=d898dc915d39e5d9f7f7148191e96cf0e8655408f79dc292c6e0b9dc27b8099a
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=d6a7c2bca29e

In [ ]:
from Modules.functions import get_labels, get_hierarchy_topic
from Modules.params import get_params
from top2vec import Top2Vec
from os.path import join
import hdbscan
import umap

In [ ]:
params = get_params()
filename = join(params["path models"],
                params["yahoo model"])
model = Top2Vec.load(filename)

In [ ]:
umap_args = {
    "n_neighbors": 15,
    "n_components": 2,
    "metric": "cosine",
}
umap_model = umap.UMAP(**umap_args)
vectors = umap_model.fit(model.document_vectors)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
cluster_model = hdbscan.HDBSCAN(min_cluster_size=15,
                                metric='euclidean',
                                cluster_selection_method='eom')
cluster = cluster_model.fit(umap_model.embedding_)
hierarchy_dict = get_hierarchy_topic(model,10)
clustered, outliers = get_labels(umap_model,
                                 model)
clustered["topic"] = clustered["doc_top"].apply(
    lambda label: hierarchy_dict[label])
filename = join(params["path results"],
                "yahoo_clustered.csv")
clustered.to_csv(filename,
                 index=False)